# problem instance

In [1]:
from __future__ import annotations

# General imports
import numpy as np

# Pre-defined ansatz circuit, operator class and visualization tools
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution

# Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

# rustworkx graph library
import rustworkx as rx
from rustworkx.visualization import mpl_draw

# SciPy minimizer routine
from scipy.optimize import minimize

In [2]:
import random

In [1]:
def define_instance(PROBLEM_DIM, instance, verbose):
    """
    Parameters
    ----------
        PROBLEM_DIM (int): problem dimension
        instance(int): number of instance
        verbose (bool): if True, print stuff
    
    """
    if PROBLEM_DIM==6:
        # ############# DIM 6 #############
        all_instances = {
        1: [{4, 6, 7, 9, 10, 11}, {1, 2, 5, 6, 11, 12}, {8, 1, 12}, {2, 3, 5}, {1, 3, 4, 5, 9, 12}, {2, 6, 7, 9, 12}] ,
        2: [{2, 11, 12, 6}, {2, 4, 6, 8, 9, 11}, {1, 3, 5, 7, 10, 12}, {2, 7}, {2, 3, 4, 5, 8, 12}, {1, 2, 8, 9, 12}] ,
        3: [{8, 10, 3}, {2, 4, 5, 6, 9, 11, 12}, {1, 3, 7, 8, 10}, {3, 4, 6, 8, 11}, {2, 3, 4, 6, 7, 9, 12}, {1, 7}] ,
        4: [{1, 4, 5, 6, 8, 11, 12}, {3, 6, 8, 9, 10}, {1, 2, 11, 5}, {2,3, 7, 9,10}, {8, 3, 12, 6}, {4, 7, 12}] ,
        5: [{11, 7}, {1, 2, 4, 5, 9, 11}, {1, 3, 4, 6, 8}, {2, 5}, {3, 6, 7, 8,10, 12}, {9, 10, 12}] ,
        6: [{2, 3, 4, 5, 7, 8,9, 10, 11, 12}, {5, 9, 10, 11}, {3, 6, 7, 8}, {1, 2, 4, 12}, {1, 6}, {2, 3, 4, 7, 8, 12}] ,
        7: [{1, 2, 5, 6, 7, 9, 12}, {10, 3, 4}, {2, 5, 7, 8, 9, 10, 11}, {8, 11}, {1, 7, 8, 11, 12}, {11, 9, 3, 7}] ,
        8: [{1, 2, 4, 5, 6, 8, 9, 10}, {1, 4, 6, 7}, { 5, 8, 9, 11, 12}, {4, 7, 8, 9, 10, 11, 12}, {2, 3, 4, 5, 6, 7, 11, 12}, {10, 2, 3}] ,
        9: [{1, 4, 7, 9, 10, 11, 12}, {2, 3, 5, 6, 8}, {1, 2, 4, 9, 12}, {2,3, 6, 7, 8, 9}, {1, 4, 6, 7, 8, 10, 11}, {8, 11, 3, 6}] ,
        10: [{1, 11, 12, 6}, {2, 5, 8, 10, 11, 12}, {2, 6, 7, 8, 9, 10, 11}, {2, 3,4, 5, 7, 8, 9, 10}, {1, 2, 4, 5, 6, 7, 8, 11}, {1, 5, 6, 9, 10, 11, 12}]} 

    elif PROBLEM_DIM==8:
        ########### DIM 8 ###########
        all_instances = { 
        1: [{1, 2, 3, 4, 7, 8, 9, 12, 13, 14}, {1, 4, 5, 8, 9, 12, 13, 14, 16}, {1, 6, 9, 10, 11, 13, 15, 16}, {5, 6, 10, 11, 15, 16}, {1, 6, 7, 8, 9, 10, 14, 15}, {8, 13, 4, 12}, {2, 3, 4, 5, 7, 8, 11, 12, 13}, {4, 5, 6, 7, 10, 12, 14, 16}],
        2: [{1, 2, 3, 5,7,12, 15}, {6, 7, 8, 9}, {4, 6, 8, 9, 10, 11, 13, 14, 16}, {3, 4, 5, 6, 7, 9, 10, 11, 12, 15}, {4, 12, 14}, {2, 3, 7, 8, 10, 15}, {1, 4, 6, 8, 9, 10, 12, 14, 15}, {1, 2, 3, 5, 10, 11,13, 15, 16}],
        3: [{4, 7, 14, 15,16}, { 4, 5, 8, 9, 10, 13, 14, 16}, {1, 3, 4, 6, 8, 9, 10, 12, 13, 14}, {1, 3, 5, 6, 10}, {2, 4, 6, 9, 10, 11, 15}, {1, 2, 4, 5, 9, 13, 14, 15}, {1, 2, 3, 6, 7, 11, 12, 15}, {2, 8, 9, 11, 12, 13}], 
        4: [{2, 4, 5, 6, 9}, {13, 14, 15, 16}, {1, 4, 5, 6}, { 7, 8, 10, 12}, {1, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16}, {2, 3, 9, 11}, {3, 4, 5, 7, 8, 10, 12}, {3, 7, 8, 9, 10, 11, 14}],
        5: [{5, 8, 10, 11, 12, 13, 16}, {1, 2, 3, 4, 6, 7, 9, 14, 15}, {1, 2, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16}, {3, 4, 9, 11, 13, 14, 16}, {2, 3, 6, 10, 13, 16}, {1, 3, 8, 10, 11, 13,14, 15}, {1, 6, 8, 10, 11, 13, 14, 16}, {3, 4, 7, 10, 11, 13, 14, 15, 16}],
        6: [{1, 2, 5, 6, 7, 10, 12, 15}, {5, 6, 9, 10}, {2, 7, 11, 12}, {3, 6, 7, 8, 9, 12, 15, 16}, {1, 3, 4, 8, 13, 14, 15, 16}, {2, 3, 6, 7, 8, 10, 15, 16}, {1, 2, 3, 4, 7, 8, 12, 13, 14}, {2, 3, 8, 11, 12, 14}],
        7: [{2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 15}, {2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16}, {10, 11, 15, 7}, {2, 3, 9, 10, 11, 14, 16}, {1, 2, 7, 10, 11, 12, 13, 14, 15}, {1, 4, 9, 14, 16}, {3, 4, 5, 9, 11, 12, 14, 16}, {2, 7, 9, 10, 11, 12, 13, 14, 15, 16}],
        8: [{1, 4, 8, 9, 10, 13, 14, 15, 16}, {1, 2, 4, 5, 6, 7, 9, 12, 13}, {2, 3, 5, 6, 7, 11, 12}, { 5, 11, 12}, {1, 2, 3, 7, 8, 10, 11, 13}, {1, 2, 4, 7, 8, 13, 14, 15, 16}, {2, 3, 6, 7}, {1, 3, 4, 6, 9, 11, 12, 14, 16}],
        9: [{2, 4, 14,15}, {1, 3, 6, 8}, {5, 7, 10, 13, 16}, {1, 2, 4, 6, 7, 10, 11, 13, 15, 16}, {4, 6, 7, 11, 12, 13, 15}, {3, 5, 8, 9, 12, 14}, {9, 11, 12}, {2, 3, 4, 5, 7, 8, 9, 10, 12, 14, 15, 16}],
        10: [{2, 3, 5, 6, 7, 8, 9, 11, 13, 16}, {1, 4, 5, 7, 9, 12, 14}, {6, 7, 11, 13, 14, 15, 16}, {6, 7, 8, 9, 11, 15, 16}, {2, 5, 10, 12, 14}, {1, 5, 6, 7, 11, 12, 14}, {2, 3, 4, 5, 7, 11, 12, 13, 14, 15, 16}, {1, 3, 4, 13}]
        }
    elif PROBLEM_DIM==10:
            ########## DIM 10 ###########
        all_instances = {
         1:[{1, 4, 7, 8, 9, 12, 13, 14, 15, 17}, {1, 3, 6, 8, 9, 13, 14, 16, 17}, 
                         {2, 4, 5, 7, 10,11}, {1, 4, 5, 8, 9, 11, 13, 14, 17, 19}, {1, 5, 6, 10, 11, 13, 14, 16, 18, 19, 20}, 
                         {1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 16}, {2, 3, 5, 6, 10, 11, 16, 18,19,20}, {12, 15, 18, 19,20}, 
                         {2, 4, 5, 6, 8, 9, 10, 11, 13, 15, 17, 19, 20}, {5, 7, 8, 10, 12, 13, 16, 17, 18, 19, 20}],                 
        2:[{1, 2, 5, 7, 8, 9, 10, 11, 12, 13, 15, 17, 20}, {2, 3, 4, 5, 9,10,20}, {1, 6, 7, 8, 11, 12, 13, 19}, { 14, 15, 16, 17, 18}, 
                         {3, 5, 6, 10, 16, 18, 20}, {4, 7, 8, 9, 12, 14, 17, 19}, {3, 4, 6, 14, 16, 18, 19}, {1, 2, 11, 13, 15}, 
                         {2, 6, 7, 10, 13, 14, 16, 17, 20}, {2, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 20}],                
        3:[{2, 3, 6, 7, 9}, {4, 5, 8, 12, 15}, {10, 11, 13, 14, 17}, {1, 16, 18, 19, 20}, {2, 10, 11, 13, 15, 16, 17, 18}, {2, 5, 18,19,20}, 
                         {3, 4, 5, 6, 7, 8, 10, 11, 16, 17, 20}, {7, 8, 10, 11, 13, 14, 16, 17}, {1, 3, 4, 6, 9, 12, 15}, 
                         {1, 2, 4, 5, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20}],                 
        4:[{1, 2, 5, 8, 10, 11, 13, 15,18, 20}, {3, 4, 6, 7, 9, 12, 14, 16, 17, 19}, {2, 5, 6, 11, 15, 17}, {1, 3, 4, 7, 8, 9, 10, 12}, 
                         {13, 14, 16, 18, 19, 20}, {2, 5, 6, 8, 9, 10, 11, 12, 14, 16, 18, 19, 20}, {1, 2, 6, 8, 10, 11, 13, 14, 15, 16, 18, 19, 20},
                         {1, 2, 3, 4, 7, 10, 16, 17}, {1, 2, 4, 5, 7, 8, 9, 11, 13, 14, 15, 17, 19}, {1, 2, 3, 4, 8, 9, 13, 15, 16, 17, 19}],                 
        5:[{2, 4, 5, 6, 9, 15, 16, 19}, {3, 7, 8, 12, 13, 14, 17}, {1, 10, 11, 18, 20}, {2, 8, 9, 10, 12, 13, 14, 15, 16, 17}, 
                         {1, 3, 4, 5, 6, 7, 11, 18, 19, 20}, {1, 3, 6, 7, 8, 9, 12, 14, 16, 19, 20}, {1, 4, 6, 10, 11, 12, 18, 19}, 
                         {1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 20}, {1, 2, 9, 11, 15}, {2, 3, 4, 5, 6, 7, 9, 10, 11, 14, 19}],                 
        6:[{1, 2, 3, 4, 9, 10, 12, 13, 15, 19, 20}, {2, 3, 5, 6, 8, 10, 12, 17}, {1, 4, 7, 9, 14, 15, 19}, {11, 13, 16,18, 20}, 
                         {2, 3, 4, 5, 6, 8, 9, 13, 14, 15,16, 17}, {1, 7, 10, 11, 12, 18, 19, 20}, {1, 2, 5, 11, 13, 14, 19, 20}, 
                         {1, 5, 7, 9, 10, 11, 13}, {12, 14, 18, 19, 20}, {2, 3, 4, 6, 8, 15, 16, 17}],
        7:[{2, 3, 6, 9, 10, 12, 14, 17, 20}, {1, 4, 10, 11, 12, 14, 15, 17, 18, 20}, {2, 4, 5, 7, 8, 11, 12, 15, 17}, 
                         {2, 3, 4, 5, 6, 8, 9, 11, 14, 17, 18, 20}, {1, 2, 4, 16, 17, 18, 20}, {1, 4, 5, 7, 8, 11, 13, 15, 16, 18, 19}, 
                         {5, 6, 8, 10, 11, 13, 15}, {3, 7, 9, 12, 14, 19}, {1, 2, 4, 6, 7, 8, 9, 12, 13, 15, 17, 18, 20}, 
                         {1, 5, 10, 12, 15, 17, 18, 19}],                 
        8:[{ 4, 8, 12, 15, 16, 17, 18}, {1, 2, 4, 7, 8, 10, 11, 12, 13}, {3, 5, 6, 12, 14, 18, 19,20}, {6, 7, 9, 10, 13, 14, 19}, 
                         {1, 2, 3, 5, 11, 20}, {1, 2, 4, 7, 8, 9, 10, 11, 13, 15, 16, 17}, {1, 3, 6, 8, 10, 15, 17, 19}, 
                         {2, 3, 4, 10, 12, 13, 14, 16, 17, 20}, {2, 3, 4, 8, 11, 12, 13, 18, 20}, {1, 6, 7, 9, 10, 11, 13, 14, 16}],
        9:[{2, 4, 5, 6, 7, 11, 15, 17}, {5, 9, 10, 14, 17, 18, 20}, {4, 5, 6, 8, 9, 11, 13, 14, 16, 17, 20}, {4, 7, 8, 11,13, 15, 19}, 
                         {2, 5, 6, 7, 12, 14, 15, 16, 17}, {1, 2, 3, 6, 12, 16}, {1, 2, 5, 6, 8, 10, 14, 16, 18, 19, 20}, 
                         {1, 3, 4, 8, 9, 10, 11, 13, 18, 19,20}, {1, 4, 5, 8, 9, 10, 11, 13, 14, 16, 18}, {1, 3, 6, 7, 8, 12, 13, 15, 16, 17, 19}],
        10:[{1, 2, 6, 7, 9, 11, 18}, {3, 4, 8, 10, 13, 19, 20}, { 5, 12, 14, 15, 16, 17}, {1, 2, 5, 6, 7, 9, 11, 12, 14, 15, 16, 17, 18}, 
                         {2, 4, 7, 8, 9, 10, 12, 14, 16, 18}, {1, 4, 7, 8, 10, 13, 18}, {2, 3, 5, 8, 9, 11, 12, 16, 20}, 
                         {1, 4, 7, 8, 11, 12, 13, 14, 18, 19, 20}, {3, 4, 5, 9, 11, 12, 13, 14, 15, 16, 17}, {2, 5, 7, 8, 13, 18, 19, 20}]
        }

    # Create a dictionary containing the subsets of the instance.
    subsets_dict = dict([(k,v) for k,v in enumerate(all_instances[instance])])
    
    # Create a list of the subsets.
    subsets = all_instances[instance]
    
    # Find U set by union of the subsets.
    U = find_U_from_subsets(subsets_dict)
    

    if verbose:
        print(f"## PROBLEM_DIM chosen: {PROBLEM_DIM} ##")
        print(f"## instance chosen: {instance} ##")
        print("subsets:\n", subsets)
        print("U:\n", U)
    
    return U, subsets_dict

In [ ]:
def build_instance_graph(subsets, verbose):
    """
    Parameters
    ----------
        subsets (dict): dictionary defining the instance
        verbose (bool): if True, print stuff
    
    """
    PROBLEM_DIM = len(subsets)
    """
    Build the graph of the instance
    """
    edges = edges_from_sets(subsets)
    G = rx.PyGraph()
    G.add_nodes_from([i for i in range(1, PROBLEM_DIM+1)])
    G.extend_from_edge_list(edges)
    # G.remove_node(0)
    mpl_draw(
        G, pos=rx.circular_layout(G), with_labels=True, node_color="#EE5396", font_color="#F4F4F4"
    )
    
    """
    Find the lengths: for every subset see
    many elements it has.
    """
    lengths = [len(s) for s in subsets]
    
    """
    Find the list of intersections: for every subset see
    with which subsets it has a non zero intersection.
    """
    list_of_intersections = find_intersections_lists(subsets)
    # for i,l in enumerate(list_of_intersections):
    #     print(f"Set #{i} intersections:", l)
    # print("list_of_intersections", list_of_intersections)
    
    """
    Find the valencies: for every subset see
    with how many subsets it has a non zero intersection.
    """
    valencies = [len(x) for x in list_of_intersections]

    if verbose:
        print("lenghts : ", lengths)
        print("valencies : ", valencies)

    return list_of_intersections

## energy spectrum

In [13]:
def find_spectrum(U, subsets_dict, PROBLEM_DIM, k):

    subsets = list(subsets_dict.values())
    states = []
    energies = []
    states_feasible = []
    energies_feasible = []

    EXACT_COVERS = []
    
    for nuple in bit_gen(PROBLEM_DIM):
        
        state = "".join([str(bit) for bit in nuple]) # strings
        chosen_subsets = [subsets[i] for i,digit in enumerate(state) if digit=="1"]
        u = set().union(*chosen_subsets)
        sum_of_len = sum([len(sub) for sub in chosen_subsets])
    
        """La somma delle lunghezze dei sottoinsiemi selezionati da uno stato
           è uguale alla lunghezza dell'insieme unione u solo 
           se i sottoinsiemi non si intersecano, 
           ovvero se lo stato soddisfa il vincolo.
        """
        energy = compute_energy_Wang(state, U, subsets_dict, k)
        
        if len(u) == sum_of_len:        
            states_feasible.append(state)
            energies_feasible.append(energy)
        
        states.append(state)
        energies.append(energy)
        
        #### CHECK IF STATE IS AN EXACT COVER
        E = compute_energy_Lucas(state, U, subsets_dict)
        if E == 0: EXACT_COVERS.append(state)
            
    return states, energies, states_feasible, energies_feasible, EXACT_COVERS


Find intersections between sets.

In [5]:
def find_intersections_lists(sets):
    """Parameters
       ----------
       sets (list of sets): sets for which we want to find intersections.

       Return
       ------
       list_of_intersections (list of lists): the i-th element of the list is 
                    a list containing the indeces of all the sets which have 
                    non-zero intersection with the i-th set of `sets`.
       
       Example
       -------
       "Set #0 intersections: [1,5]" means that the 0th set has
       nonzero intersection with the 1st and 5th set of `sets`.
                   
    """
    list_of_intersections = []
    for i in range(len(sets)):
        ith_set_intersections = []
        for j in range(len(sets)):
            if i!=j and sets[i].intersection(sets[j]):
                ith_set_intersections.append(j)
        list_of_intersections.append(ith_set_intersections)
    return list_of_intersections

# ***************************************************************

def edges_from_sets(sets):
    """Parameters
       ----------
       sets (list): a list of sets.

       Returns
       -------
       egs (list of tuples): list of tuples that will be edges of a graph
                             connecting sets that have nonzero intersection.
                             E.g. (1,3) <-> sets 1 and 3 have nonzero intersection.
    """
    egs = []
    l = len(sets)
    for i in range(l):
        for j in range(i,l):
            w = sets[i].intersection(sets[j])
            if len(w)!=0 and i!=j:
                egs.append((i,j))
                
    return egs

def find_U_from_subsets(subsets_dict):
    U = subsets_dict[0]
    for s in subsets_dict.values():
        U = U | s
    return U

# cost hamiltonian

In [6]:
# from itertools import permutations, combinations

from more_itertools import distinct_permutations

from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, QuantumRegister

# mixing hamiltonian

In [7]:
import itertools
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library.standard_gates import RXGate
from qiskit.circuit import Parameter
from qiskit.circuit.library import MCMTVChain

## Modifico il codice di IBM mettendo come controllo "0" + esempio

In [8]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2020.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""Multiple-Control, Multiple-Target Gate."""


from collections.abc import Callable

from qiskit import circuit
from qiskit.circuit import ControlledGate, Gate, QuantumRegister, QuantumCircuit
from qiskit.exceptions import QiskitError

# pylint: disable=cyclic-import
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate, HGate, TGate, TdgGate, SGate, SdgGate


class MCMT(QuantumCircuit):
    """The multi-controlled multi-target gate, for an arbitrary singly controlled target gate.

    For example, the H gate controlled on 3 qubits and acting on 2 target qubit is represented as:

    .. parsed-literal::

        ───■────
           │
        ───■────
           │
        ───■────
        ┌──┴───┐
        ┤0     ├
        │  2-H │
        ┤1     ├
        └──────┘

    This default implementations requires no ancilla qubits, by broadcasting the target gate
    to the number of target qubits and using Qiskit's generic control routine to control the
    broadcasted target on the control qubits. If ancilla qubits are available, a more efficient
    variant using the so-called V-chain decomposition can be used. This is implemented in
    :class:`~qiskit.circuit.library.MCMTVChain`.
    """

    def __init__(
        self,
        gate: Gate | Callable[[QuantumCircuit, circuit.Qubit, circuit.Qubit], circuit.Instruction],
        num_ctrl_qubits: int,
        num_target_qubits: int,
    ) -> None:
        """Create a new multi-control multi-target gate.

        Args:
            gate: The gate to be applied controlled on the control qubits and applied to the target
                qubits. Can be either a Gate or a circuit method.
                If it is a callable, it will be casted to a Gate.
            num_ctrl_qubits: The number of control qubits.
            num_target_qubits: The number of target qubits.

        Raises:
            AttributeError: If the gate cannot be casted to a controlled gate.
            AttributeError: If the number of controls or targets is 0.
        """
        if num_ctrl_qubits == 0 or num_target_qubits == 0:
            raise AttributeError("Need at least one control and one target qubit.")

        # set the internal properties and determine the number of qubits
        self.gate = self._identify_gate(gate)
        self.num_ctrl_qubits = num_ctrl_qubits
        self.num_target_qubits = num_target_qubits
        self.ctrl_state = "0" * self.num_ctrl_qubits
        
        num_qubits = num_ctrl_qubits + num_target_qubits + self.num_ancilla_qubits

        # initialize the circuit object
        super().__init__(num_qubits, name="mcmt")
        self._label = f"{num_target_qubits}-{self.gate.name.capitalize()}"

        # build the circuit
        self._build()

    def _build(self):
        """Define the MCMT gate without ancillas."""
        if self.num_target_qubits == 1:
            # no broadcasting needed (makes for better circuit diagrams)
            broadcasted_gate = self.gate
        else:
            broadcasted = QuantumCircuit(self.num_target_qubits, name=self._label)
            for target in list(range(self.num_target_qubits)):
                broadcasted.append(self.gate, [target], [])
            broadcasted_gate = broadcasted.to_gate()

        mcmt_gate = broadcasted_gate.control(self.num_ctrl_qubits, ctrl_state= self.ctrl_state)
        self.append(mcmt_gate, self.qubits, [])

    @property
    def num_ancilla_qubits(self):
        """Return the number of ancillas."""
        return 0

    def _identify_gate(self, gate):
        """Case the gate input to a gate."""
        valid_gates = {
            "ch": HGate(),
            "cx": XGate(),
            "cy": YGate(),
            "cz": ZGate(),
            "h": HGate(),
            "s": SGate(),
            "sdg": SdgGate(),
            "x": XGate(),
            "y": YGate(),
            "z": ZGate(),
            "t": TGate(),
            "tdg": TdgGate(),
        }
        if isinstance(gate, ControlledGate):
            base_gate = gate.base_gate
        elif isinstance(gate, Gate):
            if gate.num_qubits != 1:
                raise AttributeError("Base gate must act on one qubit only.")
            base_gate = gate
        elif isinstance(gate, QuantumCircuit):
            if gate.num_qubits != 1:
                raise AttributeError(
                    "The circuit you specified as control gate can only have one qubit!"
                )
            base_gate = gate.to_gate()  # raises error if circuit contains non-unitary instructions
        else:
            if callable(gate):  # identify via name of the passed function
                name = gate.__name__
            elif isinstance(gate, str):
                name = gate
            else:
                raise AttributeError(f"Invalid gate specified: {gate}")
            base_gate = valid_gates[name]

        return base_gate

    def control(self, num_ctrl_qubits=1, label=None, annotated=False):
        ctrl_state = '0' * num_ctrl_qubits
        """Return the controlled version of the MCMT circuit."""
        if not annotated and ctrl_state is None:
            gate = MCMT(self.gate, self.num_ctrl_qubits + num_ctrl_qubits, self.num_target_qubits)
        else:
            gate = super().control(num_ctrl_qubits, label, ctrl_state, annotated=annotated)
            print(ctrl_state, num_ctrl_qubits, "ciao")
        return gate

    def inverse(self, annotated: bool = False):
        """Return the inverse MCMT circuit, which is itself."""
        return MCMT(self.gate, self.num_ctrl_qubits, self.num_target_qubits)


class MCMTVChain(MCMT):
    """The MCMT implementation using the CCX V-chain.

    This implementation requires ancillas but is decomposed into a much shallower circuit
    than the default implementation in :class:`~qiskit.circuit.library.MCMT`.

    **Expanded Circuit:**

    .. plot::

       from qiskit.circuit.library import MCMTVChain, ZGate
       from qiskit.visualization.library import _generate_circuit_library_visualization
       circuit = MCMTVChain(ZGate(), 2, 2)
       _generate_circuit_library_visualization(circuit.decompose())

    **Examples:**

        >>> from qiskit.circuit.library import HGate
        >>> MCMTVChain(HGate(), 3, 2).draw()

        q_0: ──■────────────────────────■──
               │                        │
        q_1: ──■────────────────────────■──
               │                        │
        q_2: ──┼────■──────────────■────┼──
               │    │  ┌───┐       │    │
        q_3: ──┼────┼──┤ H ├───────┼────┼──
               │    │  └─┬─┘┌───┐  │    │
        q_4: ──┼────┼────┼──┤ H ├──┼────┼──
             ┌─┴─┐  │    │  └─┬─┘  │  ┌─┴─┐
        q_5: ┤ X ├──■────┼────┼────■──┤ X ├
             └───┘┌─┴─┐  │    │  ┌─┴─┐└───┘
        q_6: ─────┤ X ├──■────■──┤ X ├─────
                  └───┘          └───┘
    """

    def _build(self):
        """Define the MCMT gate."""
        control_qubits = self.qubits[: self.num_ctrl_qubits]
        target_qubits = self.qubits[
            self.num_ctrl_qubits : self.num_ctrl_qubits + self.num_target_qubits
        ]
        ancilla_qubits = self.qubits[self.num_ctrl_qubits + self.num_target_qubits :]

        if len(ancilla_qubits) > 0:
            master_control = ancilla_qubits[-1]
        else:
            master_control = control_qubits[0]

        self._ccx_v_chain_rule(control_qubits, ancilla_qubits, reverse=False)
        for qubit in target_qubits:
            self.append(self.gate.control(ctrl_state='0'), [master_control, qubit], [])
        self._ccx_v_chain_rule(control_qubits, ancilla_qubits, reverse=True)

    @property
    def num_ancilla_qubits(self):
        """Return the number of ancilla qubits required."""
        return max(0, self.num_ctrl_qubits - 1)

    def _ccx_v_chain_rule(
        self,
        control_qubits: QuantumRegister | list[circuit.Qubit],
        ancilla_qubits: QuantumRegister | list[circuit.Qubit],
        reverse: bool = False        
    ) -> None:
        """Get the rule for the CCX V-chain.

        The CCX V-chain progressively computes the CCX of the control qubits and puts the final
        result in the last ancillary qubit.

        Args:
            control_qubits: The control qubits.
            ancilla_qubits: The ancilla qubits.
            reverse: If True, compute the chain down to the qubit. If False, compute upwards.

        Returns:
            The rule for the (reversed) CCX V-chain.

        Raises:
            QiskitError: If an insufficient number of ancilla qubits was provided.
        """
        
        ctrl_state='0'*len(control_qubits)
        
        if len(ancilla_qubits) == 0:
            return

        if len(ancilla_qubits) < len(control_qubits) - 1:
            raise QiskitError("Insufficient number of ancilla qubits.")

        iterations = list(enumerate(range(2, len(control_qubits))))
        if not reverse:
            self.ccx(control_qubits[0], control_qubits[1], ancilla_qubits[0], ctrl_state='00')
            for i, j in iterations:
                self.ccx(control_qubits[j], ancilla_qubits[i], ancilla_qubits[i + 1], ctrl_state='00')
        else:
            for i, j in reversed(iterations):
                self.ccx(control_qubits[j], ancilla_qubits[i], ancilla_qubits[i + 1], ctrl_state='00')
            self.ccx(control_qubits[0], control_qubits[1], ancilla_qubits[0], ctrl_state='00')

    def inverse(self, annotated: bool = False):
        return MCMTVChain(self.gate, self.num_ctrl_qubits, self.num_target_qubits)

Piccolo esempio per vedere se il controllo è stato cambiato e se funziona:

In [9]:
# from qiskit import QuantumCircuit

# num_qubits = 10
# ctrl_qubit = 3

# qr = QuantumRegister(num_qubits, 'q')
# anc = QuantumRegister(8, 'ancilla')
# qc = QuantumCircuit(qr, anc)

# for i, bit in enumerate("0"*num_qubits):
#     qc.initialize(bit, i)

# theta = Parameter('theta') 
# gate = MCMTVChain(RXGate(theta), 9, 1).to_gate() 
# qc.append(gate, [0,1,2,4,5,6,7,8,9, ctrl_qubit, 10,11,12,13,14,15,16,17])
# qc.measure_all()
# qc.decompose().draw('mpl')

###########################################
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# from qiskit.primitives import StatevectorEstimator, StatevectorSampler
# import pandas as pd

# # Generate a pass manager without providing a backend
# pm = generate_preset_pass_manager(optimization_level=3)
# ansatz_isa = pm.run(qc)
# hamiltonian_isa = hamiltonian.apply_layout(ansatz_isa.layout)

# estimator = StatevectorEstimator()
# sampler = StatevectorSampler()

# qc = qc.assign_parameters([np.pi]) # ruoto di pi greco attorno a X
# qc_isa = pm.run(qc)
# result = sampler.run([qc_isa], shots=1024).result()
# samp_dist = result[0].data.meas.get_counts()
# samp_dist

In [1]:
def bit_gen(n):
    """
    Arguments
    ---------
       n: length of the tuples to be produced.
       
    Return
    ------
       A generator of all possible n-tuples of bits.
    """
    return itertools.product([0, 1], repeat=n)


def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    cost = result[0].data.evs[0]

    return cost
    

def compute_energy_Lucas(x, U, subsets_dict):
    """
    Arguments:
        x (str): a binary array.
        U (set): a set.
        subsets_dict (dict): a list containing subsets of U, whose union is U.

    Return:
        E : x's energy (See "Ising formulations of many NP problems" by Andrew Lucas)
    """
    # Turn the string into list: "110000" -> [1,1,0,0,0,0]
    x = [int(digit) for digit in x]

    A = 1.

    E_A = 0.
    for uu in U:
        counts = sum([x[i] for i in subsets_dict.keys() if uu in subsets_dict[i]])
        E_A += (1 - counts)**2

    E_A = A * E_A

    return E_A


def compute_energy_Wang(state, U, subsets_dict, k=1):
    """
    Arguments:
        state : a binary array.
        U : a set.
        subsets_dict : a dictionary containing subsets of U, whose union is U.
        k (int): the multiplicative factor to compute l1 = k*len(state)*l2. It must be k>0.

    Return:
        E : state's energy (See "Quantum Alternating Operator Ansatz for Solving the Minimum Exact Cover
                             Problem" by Sha-Sha Wang et al.)
    """
    # Turn the string into list: "110000" -> [1,1,0,0,0,0]
    state = [int(digit) for digit in state]
    subsets = list(subsets_dict.values())
    # print("subsets", subsets)
    
    PROBLEM_DIM = len(state)
    l2 = 1/(PROBLEM_DIM*len(U) - 2)
    l1 = k * PROBLEM_DIM * l2
    
    E = 0.
    for pos,digit in enumerate(state):
        w = len(subsets[pos])
        E += l1*w*digit - l2*digit

    E = -E
    return E

def invert_counts(counts):
        return {k[::-1]:v for k, v in counts.items()}


# # Highlight with red the exact covers
# def highlight_correct_ticks(ax, states, EXACT_COVERS):
        
#     MEC = [state for state in EXACT_COVERS if state.count("1") == min([x.count("1")  for x in EXACT_COVERS])][0]

#     ax.set_xticks(ax.get_xticks())
#     ax.set_xticklabels(labels=states, rotation=90)
#     for (state, ticklbl) in zip(states, ax.xaxis.get_ticklabels()):
#         ticklbl.set_color('cyan' if state==MEC
#                           else'red' if state in EXACT_COVERS
#                           else 'black')
# Highlight with red the exact covers

def highlight_correct_ticks(ax, EXACT_COVERS):
    
    xlabels = [elem.get_text() for elem in ax.xaxis.get_ticklabels()]
    MEC = [state for state in EXACT_COVERS if state.count("1") == min([x.count("1")  for x in EXACT_COVERS])][0]

    for (state, ticklbl) in zip(xlabels, ax.xaxis.get_ticklabels()):
        ticklbl.set_color('limegreen' if state==MEC
                          else'crimson' if state in EXACT_COVERS
                          else 'black')

# initialization & minimization

In [11]:
from datetime import datetime
import random
import time

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
import pandas as pd
import seaborn as sns

In [12]:
from qiskit import QuantumCircuit, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import random_statevector, Statevector
from qiskit_aer import Aer

import random

## file reading

In [ ]:
def define_parameters_from_filename(filename):
    """
    Parameters
    ----------
        filename (str): a string that contains ... dim6_mail5_all0_random_p3_10ra_k0.085_ ...


    Return
    ------
        PROBLEM_DIM (int): instance dimension
        instance (int): number of the instance
        init_name (str): a string describing the kind of initialization
        max_p (int): the maximum layer, the depth of the circuit
        ra (int): number of random attempts
        k (float): parameter k of the problem (l1/l2=k*PROBLEM_DIM)

    Example
    -------
        Input: "15-09@11h54m_dim6_mail3_all0_random_p3_10ra_k0.067_BOUNDS[0,2pi]xNone_pars0[0,2pi]xNone_data.txt"
        Output: 6 3 all0 3 10 0.067
    """
    
    PROBLEM_DIM = int(filename.split('dim')[1][0])
    instance = filename.split('mail')[1]
    instance = int(instance.split('_all')[0])
    init_name = filename.split(f'mail{instance}_')[1]
    init_name = init_name.split("_")[0]
    max_p = int(filename.split('random_p')[1][0])
    ra = filename.split(f'p{max_p}_')[1]
    ra = int(ra.split("ra")[0])
    k = filename.split("ra_k")[1]
    k = float(k.split("_")[0])
    return PROBLEM_DIM, instance, init_name, max_p, ra, k